In [1]:
import cv2
import os
import tensorflow as tf
import glob
import numpy as np

In [2]:
from tensorflow.python.framework.graph_util import convert_variables_to_constants

In [3]:
np.set_printoptions(threshold=1e4)

In [4]:
# tf.app.flags.DEFINE_string('train_images_path', './datasets/images/train', 'Path to training images.')
# tf.app.flags.DEFINE_string('test_images_path', './datasets/images/test', 'Path to training images.')
# tf.app.flags.DEFINE_string('model_output_path', './datasets/output/', 'Path to model checkpoint.')

# FLAGS = tf.app.flags.FLAGS

In [5]:
TRAIN_IMAGES_PATH = './datasets/images/train'
TEST_IMAGES_PATH = './datasets/images/test'
MODEL_OUTPUT_PATH = './datasets/output/'

In [6]:
def read_captcha_images(image_path):
    if not os.path.exists(image_path):
        raise ValueError('image path is not exist')
        
    images = []
    labels = []
    images_path = os.path.join(image_path, '*.jpg')
    count = 0
    for image_file in glob.glob(images_path):
        count += 1
        if count % 5000 == 0:
            print('Load {} images.'.format(count))
        image = cv2.imread(image_file)
        
        #image = image[:, :, (2, 1, 0)] # change channel
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = int(image_file.split('_')[-1].split('.')[0])
        images.append(image)
        labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [7]:
def preprocess(inputs):
    preprocessed_inputs = tf.to_float(inputs)
    preprocessed_inputs = tf.subtract(preprocessed_inputs, 128.0)
    preprocessed_inputs = tf.div(preprocessed_inputs, 128.0)
    return preprocessed_inputs

In [8]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)



In [9]:
"""cnn 模型层级：
    ·conv1
    ·conv2
    ·pool3
    ·conv4
    ·conv5
    ·pool6
    ·conv7
    ·conv8
    ·fc9
    ·fc10"""
def cnn(x):
    #[n, 28, 28, 3]
    with tf.name_scope('reshape1'):
        x_image = tf.reshape(x, [-1, 28, 28, 3])
        
    #[n, 28, 28, 32]
    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([3, 3, 3, 32])
        b_conv1 = bias_variable([32])
        layer_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    
     #[n, 28, 28, 32]
    with tf.name_scope('conv2'):
        W_conv2 = weight_variable([3, 3, 32, 32])
        b_conv2 = bias_variable([32])
        layer_conv2 = tf.nn.relu(conv2d(layer_conv1, W_conv2) + b_conv2)
        
    #[n, 14, 14, 32]
    with tf.name_scope('pool3'):
        layer_pool3 = max_pool_2x2(layer_conv2)
    
    #[n, 14, 14, 64]
    with tf.name_scope('conv4'):
        W_conv4 = weight_variable([3, 3, 32, 64])
        b_conv4 = bias_variable([64])
        layer_conv4 = tf.nn.relu(conv2d(layer_pool3, W_conv4) + b_conv4)
    
    #[n, 14, 14, 64]
    with tf.name_scope('conv5'):
        W_conv5 = weight_variable([3, 3, 64, 64])
        b_conv5 = bias_variable([64])
        layer_conv5 = tf.nn.relu(conv2d(layer_conv4, W_conv5) + b_conv5) 
    
    #[n, 7, 7, 64]
    with tf.name_scope('pool6'):
        layer_pool6 = max_pool_2x2(layer_conv5)
        
    #[n, 7, 7, 128]
    print(layer_pool6.shape)
    with tf.name_scope('conv7'):
        W_conv7 = weight_variable([3, 3, 64, 128])
        b_conv7 = bias_variable([128])
        layer_conv7 = tf.nn.relu(conv2d(layer_pool6, W_conv7) + b_conv7)
        
    #[n, 7, 7, 128]
    with tf.name_scope('conv8'):
        W_conv8 = weight_variable([3, 3, 128, 128])
        b_conv8 = bias_variable([128])
        layer_conv8 = tf.nn.relu(conv2d(layer_conv7, W_conv8) + b_conv8)
    
    #[n, 7, 7, 128]
    with tf.name_scope('reshape9'):
        layer_reshape9 = tf.reshape(layer_conv8, [-1, 7 * 7 * 128])
        
    #[n, 7, 7, 128]
    with tf.name_scope('fc10'):
        W_fc10 = weight_variable([7 * 7 * 128, 1024])
        b_fc10 = bias_variable([1024])
        layer_fc10 = tf.nn.relu(tf.matmul(layer_reshape9, W_fc10) + b_fc10)
        
    with tf.name_scope('dropout11'):
        dropout_prob = tf.placeholder(tf.float32)
        layer_dropout11 = tf.nn.dropout(layer_fc10, dropout_prob, name='prob')
        
    with tf.name_scope('fc12'):
        W_fc12 = weight_variable([1024, 10])
        b_fc12 = bias_variable([10])
        y = tf.matmul(layer_dropout11, W_fc12) + b_fc12
        
    #return y
    return y, dropout_prob

In [10]:
def loss(labels, predict_labels):
#     cross_entropy = tf.reduce_mean(
#         tf.nn.sparse_softmax_cross_entropy_with_logits(
#             logits=predict_labels, labels=labels))
    
    cross_entropy = tf.losses.sparse_softmax_cross_entropy(
            labels = labels, logits = predict_labels)
    cross_entropy = tf.reduce_mean(cross_entropy)
    return cross_entropy

In [11]:
def next_batch_set(images, labels, batch_size=128):
    """Generate a batch training data.
    
    Args:
        images: A 4-D array representing the training images.
        labels: A 1-D array representing the classes of images.
        batch_size: An integer.
        
    Return:
        batch_images: A batch of images.
        batch_labels: A batch of labels.
    """
    indices = np.random.choice(len(images), batch_size)
    batch_images = images[indices]
    batch_labels = labels[indices]
    return batch_images, batch_labels

In [16]:
def captcha_cnn():
    
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 3], name='inputs')
    y_ = tf.placeholder(tf.int32, shape=[None], name='labels')
    
    x = preprocess(x)
    
    predict_labels, dropout_prob = cnn(x)
    #predict_labels = cnn(x)
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=predict_labels, labels=y_))

    
    with tf.name_scope('adam_optimizer'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#         global_step = tf.Variable(0, trainable=False)
#         learning_rate = tf.train.exponential_decay(1e-3, global_step, 150, 0.5)
#         optimizer = tf.train.MomentumOptimizer(learning_rate, 0.5)
#         train_step = optimizer.minimize(cross_entropy, global_step)
    
    with tf.name_scope('accuracy'):
        logits = tf.nn.softmax(predict_labels)
        classes = tf.cast(tf.argmax(logits, axis=1), dtype=tf.int32, name='predict')
        classes_ = tf.identity(classes, name='classes')
        accuracy = tf.reduce_mean(tf.cast(tf.equal(classes, y_), 'float'))
        
#         correct_prediction = tf.equal(tf.argmax(predict_labels, 1), y_)
#         correct_prediction = tf.cast(correct_prediction, tf.float32)
#         accuracy = tf.reduce_mean(correct_prediction)

    train_images, train_targets = read_captcha_images(TRAIN_IMAGES_PATH)
    graph = tf.get_default_graph()
    conv1_var = graph.get_tensor_by_name('conv1/Variable:0')
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        print(sess.graph)
        print(graph)
        #print(sess.run(conv1_var))
        for i in range(301):
            batch_images, batch_labels = next_batch_set(train_images, train_targets)
            train_dict = {x: batch_images, y_: batch_labels, dropout_prob: 1}
            #train_dict = {x: batch_images, y_: batch_labels}
            sess.run(train_step, feed_dict=train_dict)
            
            if i % 100 == 0:
                loss_, acc_ , classes_, conv1_var_= sess.run([cross_entropy, accuracy, classes, conv1_var], feed_dict=train_dict)
                train_text = 'step: {}, loss: {}, acc: {}, class: {}, labels: {}, conv8: {}'.format(i, loss_, acc_, classes_, batch_labels, conv1_var_)
                print(train_text)
#                 train_accuracy = accuracy.eval(feed_dict={
#                     x: batch_images, y_: batch_labels, dropout_prob: 1.0})
#                 print('step %d, training accuracy %g' % (i, train_accuracy))
                
            #train_step.run(feed_dict={x: batch_images, y_: batch_labels, dropout_prob: 0.5})

        test_images, test_targets = read_captcha_images(TEST_IMAGES_PATH)
        print('test accuracy %g' % accuracy.eval(feed_dict={
             x: test_images, y_: test_targets, dropout_prob: 1.0}))
        #print('test accuracy %g' % accuracy.eval(feed_dict={
         #    x: test_images, y_: test_targets}))
        
        saver.save(sess, "./model/captcha_cnn_model_data")
        #把变量转换成常量
        #导出当前计算图的GraphDef部分  
#         graph = tf.get_default_graph()
#         conv8_var = sess.run(graph.get_tensor_by_name('conv8/Variable:0'))
        #conv8_read_var = sess.run(graph.get_tensor_by_name('conv8/Variable/read:0'))
        #print(sess.run(conv1_var))
        #print(conv8_read_var)
        graph_def = graph.as_graph_def()
        output_graph_def = convert_variables_to_constants(sess, graph_def, output_node_names=['accuracy/predict'])
        with tf.gfile.FastGFile('./model/captcha_cnn_model.pb', mode='wb') as f:
            f.write(output_graph_def.SerializeToString())
            print(f)

In [17]:
captcha_cnn()

(?, 7, 7, 64)
Load 5000 images.
Load 10000 images.
Load 15000 images.
Load 20000 images.
Load 25000 images.
Load 30000 images.
Load 35000 images.
Load 40000 images.
[[[[-3.26506905e-02 -5.22771254e-02  6.55576810e-02  3.64052393e-02
     1.57962888e-02 -1.27010783e-02  1.35084735e-02  7.66909346e-02
     1.53034657e-01  7.61939809e-02  1.19708061e-01  1.31399974e-01
     1.75340381e-02 -9.58248600e-03 -1.63558811e-01  2.58540194e-02
    -2.10981332e-02 -1.21776223e-01 -1.88431993e-01  8.22926611e-02
    -3.52487639e-02 -4.68931720e-02  7.78295919e-02  1.17257953e-01
     3.87191065e-02 -1.60613880e-01 -4.15005013e-02  1.48122534e-01
     1.24798536e-01  5.95655441e-02 -1.19771853e-01 -4.40943353e-02]
   [ 6.75981715e-02 -1.98748279e-02  8.82576928e-02  1.12878598e-01
    -7.27495477e-02  8.01796615e-02 -1.27687767e-01  7.60906260e-04
    -8.07634071e-02  7.96129089e-03 -1.27659321e-01  1.01662435e-01
     1.67947467e-02  1.69352908e-02  6.87546358e-02  5.77749200e-02
    -1.45959869e-0

step: 100, loss: 282.3815002441406, acc: 0.3203125, class: [5 1 2 4 3 3 5 4 9 3 3 0 6 2 2 9 6 3 8 1 8 2 0 1 6 3 2 5 0 2 7 6 9 6 3 2 5
 8 1 1 6 4 2 8 8 3 3 3 8 6 3 2 2 4 1 9 0 9 4 0 9 4 6 8 1 5 7 0 1 3 3 0 4 0
 2 2 9 0 7 5 9 4 2 2 6 2 6 9 2 1 2 3 8 2 3 8 3 5 1 3 2 5 9 3 7 3 2 6 7 2 2
 3 3 2 9 4 6 3 2 6 3 4 0 3 9 1 1 5], labels: [3 2 3 7 4 0 0 3 9 7 1 0 3 0 9 0 8 3 3 3 9 7 0 7 6 1 2 6 0 1 3 6 3 8 7 2 4
 8 1 1 8 3 9 4 0 8 2 1 6 5 1 2 7 4 7 6 1 0 4 5 9 4 9 6 1 5 3 6 9 2 2 7 4 0
 2 8 2 0 1 5 6 4 4 2 4 6 3 7 2 2 3 9 5 3 3 6 2 6 1 3 8 5 9 1 9 3 3 6 2 3 2
 6 3 6 9 4 6 8 7 4 4 7 0 2 1 0 1 6], conv8: [[[[-3.26506905e-02 -5.22771254e-02  6.55576810e-02  3.64052393e-02
     1.57962888e-02 -1.27010783e-02  1.35084735e-02  7.66909346e-02
     1.53034657e-01  7.61939809e-02  1.19708061e-01  1.31399974e-01
     1.75340381e-02 -9.58248600e-03 -1.63558811e-01  2.58540194e-02
    -2.10981332e-02 -1.21776223e-01 -1.88431993e-01  8.22926611e-02
    -3.52487639e-02 -4.68931720e-02  7.78295919e-02  1.1725795

step: 200, loss: 216.82489013671875, acc: 0.359375, class: [2 8 4 3 3 2 1 0 2 9 7 9 2 3 1 6 3 1 6 3 2 3 2 1 3 9 3 6 8 1 5 5 5 1 9 0 3
 2 8 3 5 0 2 6 2 4 3 6 8 1 3 5 1 9 4 2 0 2 9 6 6 5 9 5 6 3 5 1 2 6 5 3 7 2
 6 7 9 3 2 3 5 2 6 1 2 2 0 3 1 5 5 9 6 6 1 2 2 8 5 6 6 1 5 1 5 0 1 5 5 1 5
 2 1 0 7 2 9 2 1 1 9 0 2 1 7 9 1 6], labels: [7 3 4 3 7 2 6 3 7 6 8 1 0 3 0 9 9 7 6 3 0 3 9 3 9 3 0 8 8 9 7 5 5 8 9 9 1
 2 8 3 2 0 1 6 7 7 1 6 8 1 3 5 5 9 4 2 0 1 3 5 4 9 1 5 0 8 5 1 7 8 5 8 2 7
 6 2 7 6 2 5 5 7 6 1 5 2 0 1 4 3 7 9 6 4 3 9 8 5 3 5 6 1 0 1 9 6 1 6 6 1 8
 4 6 0 3 0 1 7 2 9 9 6 4 7 5 9 2 0], conv8: [[[[-3.26506905e-02 -5.22771254e-02  6.55576810e-02  3.64052393e-02
     1.57962888e-02 -1.27010783e-02  1.35084735e-02  7.66909346e-02
     1.53034657e-01  7.61939809e-02  1.19708061e-01  1.31399974e-01
     1.75340381e-02 -9.58248600e-03 -1.63558811e-01  2.58540194e-02
    -2.10981332e-02 -1.21776223e-01 -1.88431993e-01  8.22926611e-02
    -3.52487639e-02 -4.68931720e-02  7.78295919e-02  1.1725795

step: 300, loss: 104.34176635742188, acc: 0.5390625, class: [0 6 7 1 8 7 9 4 0 5 4 3 2 4 8 0 6 0 8 9 5 9 5 0 4 7 5 0 2 8 4 4 4 6 6 4 9
 8 7 4 4 8 6 6 7 3 5 3 7 8 8 0 9 9 1 8 6 8 5 6 1 1 8 5 0 5 9 6 4 8 9 4 4 1
 8 4 2 2 5 8 0 4 7 1 8 8 1 8 4 2 8 4 2 8 3 4 4 1 5 6 6 6 3 8 2 9 6 8 0 4 9
 7 0 8 4 4 8 8 4 3 0 3 5 4 0 3 3 4], labels: [9 3 7 1 1 2 9 4 0 5 4 5 2 4 3 0 6 4 2 9 3 4 6 0 7 1 5 0 7 6 4 4 3 6 0 4 9
 9 2 4 4 4 8 0 7 3 3 1 4 8 3 2 5 9 1 5 8 8 5 6 1 1 0 3 1 3 9 6 4 8 9 4 4 3
 8 3 2 9 3 8 9 4 7 0 0 5 3 2 4 2 9 0 2 4 6 4 9 1 5 8 6 6 3 8 7 2 5 8 0 4 1
 2 0 0 4 4 2 3 4 3 0 3 5 4 0 1 3 4], conv8: [[[[-3.26506905e-02 -5.22771254e-02  6.55576810e-02  3.64052393e-02
     1.57962888e-02 -1.27010783e-02  1.35084735e-02  7.66909346e-02
     1.53034657e-01  7.61939809e-02  1.19708061e-01  1.31399974e-01
     1.75340381e-02 -9.58248600e-03 -1.63558811e-01  2.58540194e-02
    -2.10981332e-02 -1.21776223e-01 -1.88431993e-01  8.22926611e-02
    -3.52487639e-02 -4.68931720e-02  7.78295919e-02  1.172579

Load 5000 images.
Load 10000 images.
test accuracy 0.4436
[[[[-3.26506905e-02 -5.22771254e-02  6.55576810e-02  3.64052393e-02
     1.57962888e-02 -1.27010783e-02  1.35084735e-02  7.66909346e-02
     1.53034657e-01  7.61939809e-02  1.19708061e-01  1.31399974e-01
     1.75340381e-02 -9.58248600e-03 -1.63558811e-01  2.58540194e-02
    -2.10981332e-02 -1.21776223e-01 -1.88431993e-01  8.22926611e-02
    -3.52487639e-02 -4.68931720e-02  7.78295919e-02  1.17257953e-01
     3.87191065e-02 -1.60613880e-01 -4.15005013e-02  1.48122534e-01
     1.24798536e-01  5.95655441e-02 -1.19771853e-01 -4.40943353e-02]
   [ 6.75981715e-02 -1.98748279e-02  8.82576928e-02  1.12878598e-01
    -7.27495477e-02  8.01796615e-02 -1.27687767e-01  7.60906260e-04
    -8.07634071e-02  7.96129089e-03 -1.27659321e-01  1.01662435e-01
     1.67947467e-02  1.69352908e-02  6.87546358e-02  5.77749200e-02
    -1.45959869e-01  8.02327320e-02 -7.34634474e-02 -7.42906108e-02
     3.81406210e-03  4.96252254e-02  4.80154045e-02  1.14

INFO:tensorflow:Converted 16 variables to const ops.


In [ ]:
def load_model():
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph('./model/my-model-290.meta')
        saver.restore(sess, tf.train.latest_checkpoint("model/"))

In [26]:
#test_images, test_targets = read_captcha_images(FLAGS.test_images_path)
train_images, train_targets = read_captcha_images(TRAIN_IMAGES_PATH)
test_images, test_targets = next_batch_set(train_images, train_targets)

Load 5000 images.
Load 10000 images.
Load 15000 images.
Load 20000 images.
Load 25000 images.
Load 30000 images.
Load 35000 images.
Load 40000 images.


In [63]:
test_targets

array([1, 1, 1, 7, 3, 5, 5, 4, 1, 4, 2, 5, 7, 3, 2, 0, 2, 9, 9, 0, 9, 2,
       1, 3, 7, 9, 6, 7, 9, 4, 9, 7, 4, 6, 1, 3, 9, 0, 2, 8, 2, 4, 3, 4,
       5, 1, 4, 4, 3, 1, 9, 6, 5, 0, 8, 1, 1, 3, 8, 6, 1, 0, 3, 3, 5, 9,
       8, 2, 6, 3, 7, 7, 5, 0, 8, 5, 1, 7, 4, 3, 8, 4, 2, 4, 7, 5, 4, 9,
       1, 0, 1, 3, 7, 7, 3, 3, 9, 4, 4, 9, 7, 5, 6, 4, 0, 4, 0, 4, 1, 8,
       6, 5, 3, 5, 5, 9, 4, 3, 2, 7, 7, 8, 1, 2, 1, 7, 7, 3])

In [27]:
newInput_X = tf.placeholder(tf.float32, shape=[None, 28, 28, 3])
newInput_dropKeep = tf.placeholder(tf.float32)

In [30]:
def load_graph(frozen_graph_filename):  
    # We parse the graph_def file  
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:  
        graph_def = tf.GraphDef()  
        graph_def.ParseFromString(f.read())  
  
    # We load the graph_def in the default graph  
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(  
            graph_def,
#             input_map={'inputs:0': newInput_X, 'dropout11/prob:0': newInput_dropKeep},   
#             return_elements=['accuracy/predict:0'], 
            op_dict=None,   
            producer_op_list=None)
        
    return graph

In [59]:
graph = load_graph('./model/captcha_cnn_model.pb')

In [60]:
print(graph)

In [34]:
for op in graph.get_operations():  
        print(op.name,op.values())

import/inputs (<tf.Tensor 'import/inputs:0' shape=(?, 28, 28, 3) dtype=float32>,)
import/Sub/y (<tf.Tensor 'import/Sub/y:0' shape=() dtype=float32>,)
import/Sub (<tf.Tensor 'import/Sub:0' shape=(?, 28, 28, 3) dtype=float32>,)
import/div/y (<tf.Tensor 'import/div/y:0' shape=() dtype=float32>,)
import/div (<tf.Tensor 'import/div:0' shape=(?, 28, 28, 3) dtype=float32>,)
import/reshape1/Reshape/shape (<tf.Tensor 'import/reshape1/Reshape/shape:0' shape=(4,) dtype=int32>,)
import/reshape1/Reshape (<tf.Tensor 'import/reshape1/Reshape:0' shape=(?, 28, 28, 3) dtype=float32>,)
import/conv1/Variable (<tf.Tensor 'import/conv1/Variable:0' shape=(3, 3, 3, 32) dtype=float32>,)
import/conv1/Variable/read (<tf.Tensor 'import/conv1/Variable/read:0' shape=(3, 3, 3, 32) dtype=float32>,)
import/conv1/Variable_1 (<tf.Tensor 'import/conv1/Variable_1:0' shape=(32,) dtype=float32>,)
import/conv1/Variable_1/read (<tf.Tensor 'import/conv1/Variable_1/read:0' shape=(32,) dtype=float32>,)
import/conv1/Conv2D (<tf.T

In [61]:
conv8_var = graph.get_tensor_by_name('import/conv8/Variable/read:0')

In [65]:
x = graph.get_tensor_by_name('import/inputs:0')
drop_prob = graph.get_tensor_by_name('import/dropout11/Placeholder:0') 
y = graph.get_tensor_by_name('import/accuracy/predict:0')  
          
with tf.Session(graph=graph) as sess:  
    sess.run(tf.global_variables_initializer())
    classes = sess.run(y, feed_dict={x: test_images, drop_prob:1.0})
    print(classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 2
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0
 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0]


In [36]:
with tf.Session() as sess:
    y_out = sess.run(graph, feed_dict={newInput_X: test_images, newInput_dropKeep:1.0})
    print(y_out)
    print (np.mean(np.equal(y_out, test_targets)))
    print(t)

[array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])]
0.0703125


In [13]:
np.set_printoptions(threshold = 1e6)

In [22]:
print(test_targets)

[6 2 1 5 4 7 8 4 0 6 5 8 6 4 4 1 5 5 2 6 3 1 0 4 8 0 6 0 5 3 8 2 9 2 8 4 5
 5 5 8 9 1 2 8 8 6 6 1 6 0 9 0 9 8 7 8 0 0 7 4 3 8 1 5 0 3 4 9 5 6 2 1 2 6
 1 1 5 4 4 6 5 4 6 3 6 2 1 7 1 2 4 3 2 1 6 3 9 3 6 9 1 8 0 3 5 0 7 9 4 4 4
 0 5 8 1 6 8 7 1 7 8 1 7 9 0 1 8 3 8 6 7 1 0 4 7 8 0 9 8 8 5 1 3 4 4 2 5 1
 2 8 0 6 9 9 7 6 9 0 3 7 8 1 2 8 8 5 0 1 9 7 9 8 4 5 7 4 7 1 7 0 0 9 2 9 3
 7 7 7 2 1 3 0 8 4 9 3 1 6 3 7 0 5 7 3 7 6 4 2 6 8 5 1 7 2 5 9 7 1 8 9 4 1
 8 5 4 3 3 4 8 6 9 9 1 6 7 4 9 6 2 3 7 0 1 7 7 7 7 6 7 5 7 9 3 7 4 5 8 5 2
 7 0 1 6 1 8 4 0 2 5 5 9 5 0 5 9 0 2 5 0 7 1 9 8 3 9 5 6 6 7 9 3 5 9 5 2 8
 2 0 3 6 5 5 5 6 4 9 4 6 2 3 8 0 1 0 3 3 8 0 8 1 8 9 9 8 4 5 1 3 6 9 3 7 0
 1 8 1 2 1 0 4 8 7 4 3 6 1 7 2 3 5 3 0 2 6 5 3 0 3 8 3 6 2 7 2 8 0 0 2 0 3
 0 5 1 8 2 1 5 8 1 7 5 7 8 5 5 5 3 6 9 0 4 3 5 6 5 3 1 4 1 7 6 8 2 4 3 5 3
 7 4 4 2 6 1 3 5 0 3 4 8 9 9 6 5 4 9 7 4 9 2 4 5 3 8 7 0 4 0 5 4 6 9 8 1 0
 6 3 1 4 9 1 8 7 8 8 8 1 7 7 4 8 6 5 9 8 3 0 2 4 1 6 2 1 3 5 8 9 7 7 1 4 7
 3 5 6 7 5 6 8 3 4 4 6 0 

In [55]:
for op in graph.get_operations():  
    print(op.name,op.values())
    
x = graph.get_tensor_by_name('inputs:0') 
dropout_keep_prob = graph.get_tensor_by_name('dropout11/Placeholder:0') 
y = graph.get_tensor_by_name('accuracy/predict:0') 

AttributeError: 'list' object has no attribute 'get_operations'

AttributeError: 'list' object has no attribute 'get_tensor_by_name'

In [23]:
with open('./model/captcha_cnn_model.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

    graph = tf.import_graph_def(graph_def)
    print(graph)
for op in graph.get_operations():  
        print(op.name,op.values())
    
x = graph.get_tensor_by_name('inputs:0')  
y = graph.get_tensor_by_name('accuracy/predict:0')  
test_images, test_targets = read_captcha_images(FLAGS.test_images_path)

with tf.Session(graph=graph) as sess:  
        y_out = sess.run(y , feed_dict={  
            x: test_images})  
        print(y_out) # [[ 0.]] Yay!  
        accuracy = tf.reduce_mean(tf.cast(tf.equal(classes, y_), 'float', name='accuracy'))
        accuracy_out = sess.run(accuracy)
        print (accuracy_out)  

None


AttributeError: 'NoneType' object has no attribute 'get_operations'